In [4]:
from fuzzywuzzy import process
import pandas as pd

In [8]:
liv_cond = pd.read_excel("census_results_11_6.xlsx", sheet_name=None, na_values=["-"])
repair_data = pd.read_csv("export-kr1_1-77-20240701.zip", sep=";")

In [10]:
mo = repair_data["mun_obr"].unique()

In [23]:
MO_MAPPING = {
    "Савеловское": "Савёловский район",
    "Нагорное": "Нагорный район",
    "Северное": "район Северный",
    "Восточное": "район Восточный",
    "Хорошево-Мневники": "район Хорошёво-Мнёвники",
    "Бабушкинское": "Бабушкинский район" ,
    "Поселение Троицк": "городской округ Троицк",
    "Поселение Щербинка": "городской округ Щербинка",
    "Хорошевское": "Хорошёвский район",
    "Пресненское": "Пресненский район",
    "Рязанское": "Рязанский район",
    "Измайлово": "район Измайлово"
}

parts = []
for sheet_name, data in liv_cond.items():
    mun_obr = MO_MAPPING.get(sheet_name)
    if mun_obr is None:
        mun_obr = process.extractOne(sheet_name, mo, score_cutoff=70)
        if mun_obr is None:
            print(f"Cannot find matching municipality for {sheet_name}")
            continue
        else:
            mun_obr = mun_obr[0]
    
    part = data.iloc[[17, 19, 20, 21, 22]].copy()
    part.columns = [f"c{i}" for i in range(part.shape[1])]
    part["ppf"] = (
        (
            part["c3"] + part["c4"] * 2 + part["c5"] * 3 + part["c6"] * 4 + part["c8"]
             + part["c10"] + part["c12"] + part["c14"]
        ) / part["c1"]
    )        
    part["mun_obr"] = mun_obr
    part["sheet_name"] = sheet_name
    part["n_rooms"] = [-1] + list(range(1, 5))    
    parts.append(part[["mun_obr", "ppf", "n_rooms", "sheet_name"]])

ppf_by_mo = pd.concat(parts)

Cannot find matching municipality for г. Москва
Cannot find matching municipality for Муниципальные образования ВАО
Cannot find matching municipality for Муниципальные образования ЗАО
Cannot find matching municipality for Муниципальные образования ЗелАО
Cannot find matching municipality for Муниципальные образования САО
Cannot find matching municipality for Муниципальные образования СВАО
Cannot find matching municipality for Муниципальные образования СЗАО
Cannot find matching municipality for Муниципальные образования ЦАО
Cannot find matching municipality for Муниципальные образования ЮВАО
Cannot find matching municipality for Муниципальные образования ЮЗАО
Cannot find matching municipality for Муниципальные образования ЮАО
Cannot find matching municipality for Муниципальные образования НАО
Cannot find matching municipality for Муниципальные образования ТАО


In [24]:
ppf_by_mo.head(10)

,mun_obr,ppf,n_rooms,sheet_name
17,район Богородское,2.621264,-1,Богородское
19,район Богородское,2.004463,1,Богородское
20,район Богородское,2.669125,2,Богородское
21,район Богородское,3.343224,3,Богородское
22,район Богородское,3.848692,4,Богородское
17,район Вешняки,2.703723,-1,Вешняки
19,район Вешняки,2.36266,1,Вешняки
20,район Вешняки,2.697963,2,Вешняки
21,район Вешняки,3.351128,3,Вешняки
22,район Вешняки,4.033898,4,Вешняки


In [25]:
ppf_by_mo.info()

<class 'pandas.core.frame.DataFrame'>
Index: 730 entries, 17 to 22
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   mun_obr     730 non-null    object
 1   ppf         703 non-null    object
 2   n_rooms     730 non-null    int64 
 3   sheet_name  730 non-null    object
dtypes: int64(1), object(3)
memory usage: 28.5+ KB


In [26]:
ppf_by_mo.to_csv("ppf_by_mo.csv", index=False)